In [67]:
import simtool as st
import numpy as np
import plotly.graph_objects as go
import ipywidgets as w
import math
import asyncio

# Current Progress
# Got the device Image to dynamically load. 
# Output widget working as intended
# Fixed Outputs generating multiple times 


# TO DO 
# Fix the plot labeles for the different tabs dimensions tab 
# Make the IV plot dropdown a simply ipydropdown widget
# Generate the List of output plots
# Generate test data and begin ploting to show full functionality
# Document the code for the device builder and plot builder 
# Get feedback  


# Test 

# 

In [68]:
# Name of the simtool for this workflow

simToolName = "nemo1dsim2l"

In [69]:
# Gets the simtool name and the inputs 

nb = st.utils.searchForSimTool(simToolName)
inputs = st.utils.getSimToolInputs(nb)

In [70]:
# GUI Variable to be passed to the simtool
from ipywidgets import HBox, VBox, HTML, Image, Layout, Button, ButtonStyle, Tab, BoundedIntText, BoundedFloatText, Dropdown, ToggleButton, Output, Textarea, FloatLogSlider



Energy_Band_Model = Dropdown(

    description = 'Energy Band  Model',
    value = 'Effective Mass Model',
     options=['Effective Mass Model', 'Tight-Binding Sp3s* Model', 'Tight-Binding sp3d5s* Model'],
    style = {'description_width': 'initial'},
     layout=Layout(width='450px')
)


Potential_model =  Dropdown(

    description = 'Potential Model',
    value = 'Thomas-Fermi',
     options=['Thomas-Fermi', 'Hartree'],
    style = {'description_width': 'initial'}
)


Ending_Bias = BoundedIntText(
    min = 0,
    max = 1000,
    description = 'Ending Bias (V)',
    value = 0,
    style = {'description_width': 'initial'}
)

Device = Dropdown(
   description = 'Device Type',
    value = '1-Barrier-Device',
    options=['1-Barrier-Device', '2-Barrier-Device', '3-Barrier-Device', '6-Barrier-Device'],
    style = {'description_width': 'initial'}
)


points = BoundedFloatText(
    min = 0.0,
    max = 100.0,
    description = 'No. of Points',
    value = 6,
    style = {'description_width': 'initial'}
)

Starting_bias = BoundedFloatText(
    min = 0.0,
    max = 100.0,
    value = 0,
    description = 'Starting Bias',
    style = {'description_width': 'initial'},
    tooltip = 'Test'
)

Temperature = BoundedFloatText(
    min = 77.0,
    max = 500.0,
    description = 'Temperature (K)',
    value = 300,
    style = {'description_width': 'initial'},
    tooltip="Select the model for energy band representation"

)



SC_ChargeRegion = BoundedIntText(
    min = 0,
    description = 'Semiclassical Charge Region dSC (nm)',
    value = 0,
    style = {'description_width': 'initial'}
)

EQ_Region = BoundedIntText(
    min =0,
    description = 'Equilibrium Region dEQ (nm)',
    value = 0,
    style = {'description_width': 'initial'}
)



Quantum_charge = Dropdown(
    description = 'Quantum Charge',
    value = False,
    options = [True,False],
    style = {'description_width': 'initial'}
)

Lattice_Constant = BoundedIntText(
    
    description = 'Lattice Constant (nm)',
    value = 0.283,
    style = {'description_width': 'initial'}
)

Not_Normalized_Current_Plot = Dropdown(
    description = 'Not-Normalized Current Plot',
    value = False,
    options = [True, False],
    style = {'description_width': 'initial'}
)

Scatter_Plot = Dropdown(
    description = 'Scatter Plot',
    value = False,
    options = [True, False],
    style = {'description_width': 'initial'}
)

Carrier_Surface_Distrubution_Plot = Dropdown(
    description = 'Carrier Surface Distribution Plot',
    value = False,
    options = [True, False],
    style = {'description_width': 'initial'}
)


Resivoir_Relaxation_Model = Dropdown(
    description = 'Resivoir Relaxation Model',
    value = 'Energy Independant',
    options = ['Energy Independant','Exponentialy Damped', 'Lorentzian Damped'],
    style = {'description_width': 'initial'}
)


Resivoir_Relaxation_Energy = BoundedIntText(
    min =0,
    description = 'Resivoir Relaxation Energy (meV)',
    value = 0,
    style = {'description_width': 'initial'}
)

Decay_Length= BoundedIntText(
    min =0,
    description = 'Decay Length (meV)',
    value = 0,
    style = {'description_width': 'initial'}
)

Resonance_Finder = Dropdown(
    description = 'Resonance Finder',
    value = False,
    options = ['True',False],
    style = {'description_width': 'initial'}
)


No_Homogenous_Grid_Points= BoundedIntText(
    min =0,
    description = 'No. Homogenous Grid Points',
    value = 0,
    style = {'description_width': 'initial'}
)

No_Points_per_res= BoundedIntText(
    min =0,
    description = 'No. Points Per Resonance',
    value = 0,
    style = {'description_width': 'initial'}
)

No_Points_per_EC_EF= BoundedIntText(
    min =0,
    description = 'No. Points per EC EF',
    value = 0,
    style = {'description_width': 'initial'}

)

Lancos_Iteration_Step_Size= BoundedIntText(
    min =0,
    description = 'Lancos Iteration Step Size (eV)',
    value = 0,
    style = {'description_width': 'initial'}
)


Lancos_Iteration_Limit= BoundedIntText(
    min =0,
    description = 'Lancos Iteration Limit',
    value = 0,
    style = {'description_width': 'initial'}
)

Newton_Iteration_Step_Size= BoundedIntText(
    min =0,
    description = 'Newton Iteration Step Size (eV)',
    value = 0,
    style = {'description_width': 'initial'}
)


Newton_Solver_Convergance_Condition= BoundedIntText(
    description = 'Newton Solver Convergance Condition (eV)',
    value = 0,
    style = {'description_width': 'initial'}
)



w_output = Output()

outputContainer = VBox()

In [71]:
import copy

# Template for the plots

layout_template = {
    'title' : {
        'font' : {
            'family' : 'Courier New, monospace',
            'size' : 12,
            'color' : 'Black'
        }      
    },
    'margin' : {
        't' : 80,
        'b' : 150,
        'r' : 100,
        'l' : 150
    },
    'showlegend' : True,
    'font' : {
        'family' : 'Courier New, monospace',
        'size' : 24,
        'color' : 'Black'
    },
    'yaxis' : {   
         'title' : ""
    },
    'xaxis' : {
        'title' : ""
    },
    'legend' : {
        'orientation' : 'h',
        'yanchor' : 'bottom',
        'y' : 1.0,
        'xanchor' :'right',
        'x' : 1
    },
    'template' : 'presentation'
}

In [72]:
# This function plots the IV plot from NEMO 
# Room to improve data flow of the read function 
# Will need to update for use with JSON format

def  IVPlot(r):
    # Extract the data dictionary
    data = r.read('IV')  # Returns a dictionary {'x': ..., 'y': ...}
    
    # Extract voltage (x) and current (y) from the dictionary
    x = data['x']  # Tuple of voltage values
    y = data['y']  # Tuple of current values
    
 
    # Debugging: Print to verify correct extraction
    print("Voltage (x):", x)
    print("Current (y):", y)
    
    ## Creating the plot 

    # Create a Plotly Scatter trace
    trace = go.Scatter(x=x, y=y, mode='lines', name='IV Curve')
    
    # Define the layout for the plot
    # Layout is using the template
    
    layout = copy.deepcopy(layout_template) ####### Change back to figure widget might be affected by negishi outage
    # Create and return the Plotly FigureWidget
    return go.FigureWidget(data=[trace], layout=layout)


In [73]:

def plot_IV(r):
    
    data = r.read("IV")
    voltages=data['Voltage']
    currents = data ['Current']
    # Create a plot with the base layout
    fig = go.Figure()

    # Add trace to the figure
    fig.add_trace(go.Scatter(
        x=voltages,
        y=currents,
        mode='lines',
        name="IV Curve"
    ))

    # Custom layout for this specific plot (initially set to log scale)
    fig.update_layout(
        layout_template,  # Use the base template
        yaxis=dict(
            title="Current (A)",  # Custom title for Y-axis
            type='log',  # Apply log scale to Y-axis for this plot
          
           
        ),
        xaxis=dict(
            title="Voltage (V)"  # Custom title for X-axis
        ),
        title="IV Curve",
        
        # Add buttons to toggle between linear and log scale
        updatemenus=[
            {
                'buttons': [
                    {
                        'label': 'Log Scale',
                        'method': 'relayout',
                        'args': ['yaxis.type', 'log']  # Set Y-axis to log scale
                    },
                    {
                        'label': 'Linear Scale',
                        'method': 'relayout',
                        'args': ['yaxis.type', 'linear']  # Set Y-axis to linear scale
                    }
                ],
                'direction': 'down',  # Place the button below the plot
                'showactive': True,
                'x': -0.05,  # Button position relative to the plot
                'xanchor': 'left',
                'y': -0.35,  # Button position relative to the plot
                'yanchor': 'bottom'
            }
        ]
    )
    return fig.show()

    # Show the plot
    #fig.show()



In [74]:
def plot_band(r):
    # Read the band data
    data = r.read('Band')
    voltages = list(data['Voltage (V)'].keys())

    # Create the figure widget
    fig = go.FigureWidget()

    # Function to update the plot for a single voltage
    def update_single_voltage(change):
        fig.data = []  # Clear existing traces
        voltage = voltages[slider.value]
        voltage_data = data['Voltage (V)'][voltage]
        position = voltage_data['Position (nm)']
        energy = voltage_data['Energy (eV)']
        
        # Add new trace for the selected voltage
        trace = go.Scatter(
            x=position,
            y=energy,
            mode='lines',  # Line-only plot
            name=f"Voltage {voltage} V"
        )
        fig.add_trace(trace)
        
        # Update layout with your existing template and axis titles
        fig.update_layout(
            title=f"Band Structure for Voltage {voltage} V",
            xaxis=dict(title="Position (nm)"),
            yaxis=dict(title="Energy (eV)"),
            margin=layout_template['margin'],
            font=layout_template['font'],
            legend=layout_template['legend'],
            template=layout_template['template']
        )

    # Function to plot all voltages simultaneously
    def plot_all_voltages(change=None):
        fig.data = []  # Clear existing traces
        for voltage in voltages:
            voltage_data = data['Voltage (V)'][voltage]
            position = voltage_data['Position (nm)']
            energy = voltage_data['Energy (eV)']

            # Add a trace for each voltage
            trace = go.Scatter(
                x=position,
                y=energy,
                mode='lines',  # Line-only plot
                name=f"Voltage {voltage} V"
            )
            fig.add_trace(trace)
        
        # Update layout with your existing template and axis titles
        fig.update_layout(
            title="Band Structure for All Voltages",
            xaxis=dict(title="Position (nm)"),
            yaxis=dict(title="Energy (eV)"),
            margin=layout_template['margin'],
            font=layout_template['font'],
            legend=layout_template['legend'],
            template=layout_template['template']
        )

    # Dropdown menu to select plot mode
    dropdown = widgets.Dropdown(
        options=['Single Voltage', 'All Voltages'],
        value='Single Voltage',
        description='Plot Mode:',
    )

    # Slider widget for single voltage selection
    slider = widgets.IntSlider(
        value=0, 
        min=0, 
        max=len(voltages) - 1, 
        step=1, 
        continuous_update=True,
        readout=False,
    )
    
    # Create a play/pause button
    play_button = widgets.ToggleButton(
        value=False,
        description='',
        button_style='',
        tooltip='Play/Pause',
        icon='play'
    )

    # Function to update play button icon
    def update_button_icon(change):
        play_button.icon = 'pause' if change['new'] else 'play'
        

    play_button.observe(update_button_icon, 'value')

    # Async function to handle play/pause
    async def play_loop():
        while play_button.value:
            current_index = slider.value
            next_index = (current_index + 1) % len(voltages)
            slider.value = next_index
            await asyncio.sleep(0.35)

    # Monitor play button state
    async def monitor_play_button():
        while True:
            if play_button.value:
                await play_loop()
            await asyncio.sleep(0.1)

    # Voltage bias label widget
    voltage_label = widgets.Label(value=f"Voltage Bias: {voltages[slider.value]} V")

    # Update voltage label when slider changes
    def update_voltage_label(change):
        voltage_label.value = f"Voltage Bias: {voltages[slider.value]} V"

    slider.observe(update_voltage_label, names='value')

    # Handle dropdown menu changes
    # Event handler for dropdown selection
    def on_dropdown_change(change):
        if dropdown.value == 'Single Voltage':
            slider.disabled = False  # Enable the slider
            slider.layout.display = 'flex'  # Show the slider
            voltage_label.layout.display = 'flex'  # Show the voltage label
            
            # Reset play button state
            play_button.disabled = False
            play_button.value = False  # Stop animation
            play_button.icon = 'play'  # Reset icon
           
            play_button.layout.display = 'flex'
            # Set the play button to have a centered icon within the button
            play_button.layout = widgets.Layout(
                display='flex',  # Display the button with flex layout
                justify_content='center',  # Center the icon horizontally
                align_items='center',  # Center the icon vertically
                
)

            update_single_voltage(None)
        else:
            slider.disabled = True  # Disable the slider
            slider.layout.display = 'none'  # Hide the slider
            voltage_label.layout.display = 'none'  # Hide the voltage label
            play_button.layout.display = 'none'
            # Stop the animation immediately
            play_button.value = False  # Stop animation
            plot_all_voltages()

    # Attach event listeners
    slider.observe(update_single_voltage, names='value')
    dropdown.observe(on_dropdown_change, names='value')


    # Initial plot setup
    update_single_voltage(None)

    # Stack widgets
    centered_slider = HBox([play_button,slider, voltage_label ])
    dropdown_box = HBox([dropdown])
    
    # Display widgets and figure
    display(VBox([fig, centered_slider, dropdown_box]))

    # Start monitoring the play button
    asyncio.create_task(monitor_play_button())


In [75]:


def combine():
    band_output = Output()
    iv_output = Output()

    # Function to update band plot inside Output widget
    def update_band_plot(change):
        with band_output:
            # Clear previous output
            band_output.clear_output(wait=True)
            fig = plot_band(r)  # Get the band plot figure
            #fig.show()  # This will display the figure within the band_output

    # Function to update IV plot inside Output widget
    def update_iv_plot(change):
        with iv_output:
            # Clear previous output
            iv_output.clear_output(wait=True)
            fig = plot_IV(r)  # Get the IV plot figure
            

    # Layout for the left and right halves of the plot
    left_panel = VBox([band_output], layout=widgets.Layout(width="50%"))
    right_panel = VBox([iv_output], layout=widgets.Layout(width="50%"))

    # Final layout: HBox with left and right panels
    final_layout = HBox([left_panel, right_panel])

    # Display the layout
    display(final_layout)

    # Initially plot the band and IV plot when the widget is first shown
    update_band_plot(None)  # First update to the band plot
    update_iv_plot(None)    # First update to the IV plot



In [76]:
# Function to draw the device with adjustable perspective and barriers
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import ipywidgets as widgets
from PIL import Image
from io import BytesIO

# Function to draw the device with adjustable perspective and barriers
def draw_device_with_barriers(num_barriers=1, scale=5):
    # Define the structure of the device
    layer_names = ["Contact", "Unnamed"]  # Initial layers
    layer_colors = ["#C0C7FF", "#E0E6FF"]  # Updated lighter tones for Unnamed layer
    layer_widths = [0.5, 0.25]  # Adjusted layer width
    layer_depth = 0.4 * scale  # Adjusted depth of the device
    layer_height = 0.2 * scale  # Adjusted height of the device

    # Add the barriers alternating "B" and "W" layers
    for i in range(num_barriers):
        layer_names.append("B")  # Add a "B" section
        layer_colors.append("#3A5FCD")  # Original color for B layers (changed to #3A5FCD)
        layer_widths.append(0.25)  # Barrier layer width
        if i < num_barriers - 1:  # Add a "W" section after each "B" except the last one
            layer_names.append("W")
            layer_colors.append("#E0E6FF")  # Color for W layers (same as "Unnamed")
            layer_widths.append(0.25)  # Barrier layer width

    # Add the final Unnamed layer and the last Contact layer
    layer_names.append("Unnamed")
    layer_colors.append("#E0E6FF")
    layer_widths.append(0.25)

    layer_names.append("Contact")
    layer_colors.append("#C0C7FF")  # Same color as initial "Contact"
    layer_widths.append(0.5)

    # Total device dimensions, adjusted by the scale factor
    total_length = sum(layer_widths) * (1 / scale)
    total_depth = layer_depth * (1 / scale)
    total_height = layer_height * (1 / scale)

    # Hard-code the elevation and azimuth
    elev = 15
    azim = -90

    # Create the figure with a smaller size (adjusted figure window size)
    fig = plt.figure(figsize=(25, 25))  # Smaller figure size
    
    # Create 3D axes object directly without using subplot
    ax = fig.gca(projection="3d")
    
    # Current position tracker
    current_position = 0
    
    for i, (name, color, width) in enumerate(zip(layer_names, layer_colors, layer_widths)):
        # Scale width and calculate the coordinates for the rectangular prism (section of the device)
        width_scaled = width * scale
        x = [
            current_position, 
            current_position + width_scaled, 
            current_position + width_scaled, 
            current_position,
        ]
        y = [0, 0, total_depth, total_depth]
        z = [0, 0, total_height, total_height]
        
        # Draw the top face
        ax.add_collection3d(
            Poly3DCollection(
                [list(zip(x, y, [total_height] * 4))], 
                color=color, 
                edgecolor="black", 
                linewidths=1.2  # Increased edge thickness
            )
        )
        
        # Draw the front face
        ax.add_collection3d(
            Poly3DCollection(
                [list(zip(x, [0, 0, 0, 0], z))], 
                color=color, 
                edgecolor="black", 
                alpha=0.9
            )
        )
        
        # Add label above the section in 2D coordinates with increased font size
        ax.text(
            current_position + width_scaled / 2,  # Center of the layer, scaled
            total_depth + 0.1,  # Slightly above the block
            total_height + 0.1,  # Above the block
            name if name != "Unnamed" else "",  # Avoid showing "Unnamed"
            ha="center",
            va="center",
            fontsize=24,  # Font size adjusted
            color="black",
        )
        
        # If it's the last layer, add the square end cap
        if i == len(layer_names) - 1:
            end_x = [current_position + width_scaled] * 4  # Scale the end cap width
            end_y = [0, total_depth, total_depth, 0]
            end_z = [0, 0, total_height, total_height]
            
            # Draw the square end cap
            ax.add_collection3d(
                Poly3DCollection(
                    [list(zip(end_x, end_y, end_z))],
                    color=color,  # Same color as the final layer
                    edgecolor="black",
                    linewidths=1.2  # Increased edge thickness
                )
            )
        
        # Move to the next position
        current_position += width_scaled  # Scale the position for the next layer
    
    # Adjust the view for the flattened horizontal perspective
    ax.view_init(elev=elev, azim=azim)
    
    # Auto-scale to fit the object in the window with smaller padding
    ax.set_xlim(0, total_length * 1.2)  # Adjusted width
    ax.set_ylim(-total_depth * 0.5, total_depth * 1.5)  # Adjusted depth range
    ax.set_zlim(0, total_height * 1.5)  # Adjusted height range
    
    # Set the aspect ratio dynamically based on the size of the object
    ax.set_box_aspect([total_length*2, total_depth*2, total_height*2])  # Match aspect ratios to the device
    fig.patch.set_edgecolor('black')  # Set border color
    fig.patch.set_linewidth(2) 
    
    # Hide the axes
    ax.set_axis_off()  # This hides the axis
    
    # Save the plot as a PNG file in the current directory
    img_path = "device_plot.png"  # Save in the current directory
    plt.savefig(img_path, format="png", bbox_inches='tight', pad_inches=0.1)
    plt.close(fig)
    
    # Crop the image to remove white space with hardcoded values
    cropped_image_data = crop_image_hardcoded(img_path)
    
   
    
    return(cropped_image_data)
    # Show the plot
    #plt.show()

# Function to crop the image by removing hardcoded margins
def crop_image_hardcoded(image_path):
    # Open the image using PIL
    img = Image.open(image_path)
    print("Called")
    # Hardcoded crop: Remove 100 pixels from top and bottom, 100 pixels from left and right
    left = 0
    top = 500
    right = img.width - 150
    bottom = img.height - 500
    
    # Perform the cropping
    img_cropped = img.crop((left, top, right, bottom))
    
    # Convert the cropped image to PNG format in memory and return it as bytes
    img_byte_array = BytesIO()
    img_cropped.save(img_byte_array, format='PNG')
    img_byte_array.seek(0)  # Rewind the byte stream
    
    return img_byte_array.read()  # Return the image data as bytes



In [77]:
# This will call the desired plot based on what is selected after the simulation is run


def DisplayOutput(r, plot, output): # This is the section that is not implemented yet
    output.clear_output()
    with output:
        if (plot == "IV plot"):
            output.clear_output()
            display(plot_IV(r))
            
        elif (plot == "Band"):
            output.clear_output()
            display(plot_band(r))
             
        elif (plot == "IV + Band"):
             output.clear_output()
             display(combine())    
        else:
            output.clear_output()

In [78]:
# This is the run function that will get all the inputs from the user to the simtool

r = None

def RunST():
    w_output.clear_output()
    with w_output:
        global r
        # Update inputs with values from the new widgets
        inputs.Energy_Band_Model.value = Energy_Band_Model.value
        inputs.Potential_model.value = Potential_model.value
        inputs.Ending_Bias.value = Ending_Bias.value
        inputs.Device.value = Device.value
        inputs.points.value = points.value
        inputs.Starting_bias.value = Starting_bias.value
        inputs.TEMPERATURE.value = Temperature.value
        inputs.SC_ChargeRegion.value = SC_ChargeRegion.value
        inputs.EQ_Region.value = EQ_Region.value
        inputs.Quantum_charge.value = Quantum_charge.value
        inputs.Lattice_Constant.value = Lattice_Constant.value
        #inputs.Not_Normalized_Current_Plot.value = Not_Normalized_Current_Plot.value
        inputs.Scatter_Plot.value = Scatter_Plot.value
        inputs.Carrier_Surface_Distrubution_Plot.value = Carrier_Surface_Distrubution_Plot.value
        inputs.Resivoir_Relaxation_Model.value = Resivoir_Relaxation_Model.value
        inputs.Resivoir_Relaxation_Energy.value = Resivoir_Relaxation_Energy.value
        inputs.Decay_Length.value = Decay_Length.value
        inputs.Resonance_Finder.value = Resonance_Finder.value
        inputs.No_Homogenous_Grid_Points.value = No_Homogenous_Grid_Points.value
        inputs.No_Points_per_res.value = No_Points_per_res.value
        inputs.No_Points_per_EC_EF.value = No_Points_per_EC_EF.value
        inputs.Lancos_Iteration_Step_Size.value = Lancos_Iteration_Step_Size.value
        inputs.Lancos_Iteration_Limit.value = Lancos_Iteration_Limit.value
        inputs.Newton_Iteration_Step_Size.value = Newton_Iteration_Step_Size.value
        #inputs.Newton_Solver_Convergance_Condition.value = Newton_Solver_Convergance_Condition.value Current Bug this is no working 
        print(inputs)
        r = st.Run(nb, inputs)
        outputDisplay = Dropdown(
            description = "Display Output",
            options=[
                
                "Band",
                "IV plot",
                "IV + Band"
            ],
            style = {'description_width': 'initial'},
            value = "IV plot"
        )
        outputDisplay.observe(lambda b : DisplayOutput(r, b['new'],w_output), 'value')
        outputContainer.children=[outputDisplay]
        DisplayOutput(r, outputDisplay.value, w_output)
        # Run the simulation with updated inputs
       

In [79]:


def create_custom_plot_with_barriers(num_barriers=1):
    # Set default y-positions as per your request
    y_positions = [30]  # Start with the first "Contact" layer at y=30
    layer_names = ["Contact"]  # Start with the first "Contact"
    
    # Add the second "U" layer
    y_positions.append(5)  # Position for "U" layer
    layer_names.append("U")  # Add "U" layer

    # Add alternating "B" and "W" layers based on num_barriers argument
    for i in range(num_barriers):
        y_positions.append(4)  # Position for "B" layer
        layer_names.append(f"B{(i + 1)}")  # Add "B" layer with numbering
        if i < num_barriers - 1:  # Add "W" after each "B" except the last one
            y_positions.append(5)  # Position for "W" layer
            layer_names.append("W")  # Add "W" layer

    # Add the last "U" layer
    y_positions.append(5)  # Position for second "U"
    layer_names.append("U")  # Last "U" layer

    # Add the last "Contact" layer
    y_positions.append(30)  # Position for the last "Contact"
    layer_names.append("Contact")  # Last Contact layer

    # Color mapping based on layer names
    color_map = {
        "Contact": "#C0C7FF",  # Contacts in light blue
        "U": "#E0E6FF",        # Unnamed layers (U) in light gray
        "B": "#3A5FCD",        # Barriers in blue
        "W": "#E0E6FF",        # W layers in light gray
    }

    # Create the plot
    fig = go.FigureWidget()

    # Function to update the y-position of a line
    def update_line_y_position(index, new_y):
        try:
            y_positions[index] = new_y
            update_plot()
        except ValueError:
            print(f"Invalid input for Layer {index+1}. Please enter a valid number.")

    # Function to update the plot with new positions
    def update_plot():
        # Clear previous plot data and annotations
        fig.data = []
        fig.layout.annotations = []  # Clear previous annotations
        
        x_start = 0
        for i, (y, name) in enumerate(zip(y_positions, layer_names)):
            # Scale the width based on the y-position (or the size of the layer)
            layer_width = max(0.1, y / 30.0)  # Example: Scale width proportionally to y-position
            
            x_end = x_start + layer_width  # Set width based on the layer width
            fig.add_trace(
                go.Scatter(
                    x=[x_start, x_end],
                    y=[y, y],
                    mode="lines",
                    line=dict(width=2, color=color_map.get(name, "#000000")),  # Apply color from the map
                )
            )
            fig.add_annotation(
                x=(x_start + x_end) / 2,
                y=y,
                text=f"{y} nm",  # Label each layer with its name and size
                showarrow=False,
                font=dict(color="black", size=12),
                yshift=20,  # Position the label above the line, further to prevent overlap
            )
            x_start = x_end  # Move start position for the next line

        fig.update_layout(
            xaxis=dict(visible=False),  # Hide x-axis
            yaxis=dict(title="Dimensions (nm)", showgrid=False, zeroline=False),
            plot_bgcolor="white",  # White background
            margin=dict(l=50, r=50, t=50, b=50),
            showlegend=False,
        )

    update_plot()

    # Create text input fields for each line to control the y-position for Dimension, Doping, and Mole Fraction
    input_boxes = {"Dimension": [], "Doping": [], "Mole Fraction": []}
    update_thread = None  # Initialize update_thread to be used in the nonlocal scope

    # Logic for assigning labels (Contact, U, B, W) based on the index
    for i in range(len(layer_names)):
        description = f"{layer_names[i]}"

        input_box_dim = widgets.Text(
            value=str(y_positions[i]),
            description=f"Dim-{layer_names[i]}",
            continuous_update=True,  # Ensures the input updates continuously
        )

        input_box_dop = widgets.Text(
            value=str(y_positions[i] * 1e16),  # Example doping value, can be adjusted
            description=f"Dop-{layer_names[i]}",
            continuous_update=True,
        )

        input_box_mf = widgets.Text(
            value=str(y_positions[i] / 30.0),  # Example mole fraction value, can be adjusted
            description=f"MF-{layer_names[i]}",
            continuous_update=True,
        )

        # Add input boxes to the corresponding tabs
        input_boxes["Dimension"].append(input_box_dim)
        input_boxes["Doping"].append(input_box_dop)
        input_boxes["Mole Fraction"].append(input_box_mf)

        # Update the plot after a delay when the user stops typing
        def on_value_change(change, index=i, tab="Dimension"):
            nonlocal update_thread
            # Cancel the previous update if the user is typing continuously
            if update_thread is not None and update_thread.is_alive():
                update_thread.join()

            def delayed_update():
                try:
                    new_value = float(change.new)  # Try to convert the input to a float
                    update_line_y_position(index, new_value)
                except ValueError:
                    print(f"Invalid input for Layer {index+1}. Please enter a valid number.")

            # Start a new thread to update after a short delay
            update_thread = threading.Thread(target=delayed_update)
            update_thread.start()

        # Observe value change for all tabs
        input_box_dim.observe(lambda change: on_value_change(change, i, "Dimension"), names='value')
        input_box_dop.observe(lambda change: on_value_change(change, i, "Doping"), names='value')
        input_box_mf.observe(lambda change: on_value_change(change, i, "Mole Fraction"), names='value')

    # Create a container for each line and its corresponding input box for all tabs
    row_widgets = {"Dimension": [], "Doping": [], "Mole Fraction": []}
    for tab in input_boxes:
        for input_box in input_boxes[tab]:
            row_widgets[tab].append(HBox([input_box]))  # Create a horizontal box for the input field

    # Create a vertical layout for all the input boxes for each tab
    input_layout = {"Dimension": VBox(row_widgets["Dimension"]), 
                    "Doping": VBox(row_widgets["Doping"]), 
                    "Mole Fraction": VBox(row_widgets["Mole Fraction"])}

    # Function to update the Y-axis title based on the selected tab
    def update_yaxis_title(tab_index):
        if tab_index == 0:
            return "Dimensions (nm)"
        elif tab_index == 1:
            return "Doping Concentration (cm^-3)"
        elif tab_index == 2:
            return "Mole Fraction"
        return "Dimensions (nm)"

    # Create a container for the tabbed layout
    tab_layout = Tab()

    # Define the three tabs with different Y-axis titles
    tab_layout.children = [
        VBox([fig, input_layout["Dimension"]]), 
        VBox([fig, input_layout["Doping"]]), 
        VBox([fig, input_layout["Mole Fraction"]])
    ]  # Three tabs with the plot and inputs

    # Set tab titles and corresponding Y-axis titles
    tab_layout.set_title(0, 'Dimension')
    tab_layout.set_title(1, 'Doping')
    tab_layout.set_title(2, 'Mole Fraction x')

    # Update the Y-axis title dynamically when switching between tabs
    tab_layout.observe(lambda change: fig.update_layout(
        yaxis=dict(title=update_yaxis_title(change['new']))), names='selected_index')

    # Getter function to retrieve values from all the tabs
    def get_layer_values():
        values = {
            "Dimension": {},
            "Doping": {},
            "Mole Fraction": {}
        }
        for tab in input_boxes:
            for i, input_box in enumerate(input_boxes[tab]):
                if tab == "Dimension":
                    values["Dimension"][layer_names[i]] = float(input_box.value)
                elif tab == "Doping":
                    values["Doping"][layer_names[i]] = float(input_box.value)
                elif tab == "Mole Fraction":
                    values["Mole Fraction"][layer_names[i]] = float(input_box.value)
        return values

    # Example of using the getter function
    print("Current values:")
    print(get_layer_values())

    # Create an Output widget to display the plot and inputs
    output_widget = Output()
    with output_widget:
        # Display the layout and update dynamically
        display(tab_layout)

    # Return the layout and getter function
    return output_widget, get_layer_values
# Call the function to create the plot and get the Output widget



In [80]:
# Create the button and the event handler for running the simulation


scale =1
num_barriers=0
# Display the output widget in the Jupyter notebook

Image_widget = widgets.Image(value=b'', format='png')
output_widget, get_layer_values = create_custom_plot_with_barriers(num_barriers)

def update_device_image(change=None):
    # Print "click" each time the dropdown value changes
    print("click")
    
    # Use a global Image_widget to ensure the image is updated
    if Device.value == '1-Barrier-Device':
        num_barriers = 1
    elif Device.value == '2-Barrier-Device':
        num_barriers = 2  # Based on your logic, set the number of barriers
    elif Device.value == '3-Barrier-Device':
        num_barriers = 3
    elif Device.value == '6-Barrier-Device':
        num_barriers = 6

    # Generate image data based on the number of barriers
    img_data = draw_device_with_barriers(num_barriers, scale=1)
    # Update the Image widget with the new image data
    Image_widget.value = img_data
    
    
def update_widgetr(change=None):
    # Print "click" each time the dropdown value changes
    print("click")
    
    # Use a global Image_widget to ensure the image is updated
    if Device.value == '1-Barrier-Device':
        num_barriers = 1
    elif Device.value == '2-Barrier-Device':
        num_barriers = 2  # Based on your logic, set the number of barriers
    elif Device.value == '3-Barrier-Device':
        num_barriers = 3
    elif Device.value == '6-Barrier-Device':
        num_barriers = 6

 
    output_widget = create_custom_plot_with_barriers(num_barriers)
    
# Attach the update function to the dropdown 'value' change event
update_device_image()
Device.observe(update_device_image, names='value')


# Create the output area for displaying the image of the device
output_image_area = VBox(layout=Layout(width='auto', height='auto', border='1px solid rgba(0, 0, 0, 1)'))
output_image_area.children = [Image_widget]
# Create the button and the event handler for running the simulation
run_But = Button(
    description="Simulate ...", 
    tooltip='Run simulation', 
    icon='check', 
    disable=False,
    layout=Layout(width='auto'),
    style=ButtonStyle(button_color='lightblue')
)

run_But.on_click(lambda b: RunST())  # Define the action for the button click

# Create the always visible widgets section (Device and Energy Band Model)
always_visible_widgets = VBox(
    children=[
        Device,  # Add device dropdown to the always visible section
        Energy_Band_Model  
    ],
)

# Define the parameters for each tab
parameters_basic = [
    Temperature,
    Starting_bias,
    Ending_Bias,
    points,
    Potential_model,
    run_But
]

parameters_domains = [
    SC_ChargeRegion,
    EQ_Region,
    run_But
]

parameters_advanced = [    
    Not_Normalized_Current_Plot,
    Scatter_Plot,
    Carrier_Surface_Distrubution_Plot,
    Resivoir_Relaxation_Model,
    Resivoir_Relaxation_Model,
    Resivoir_Relaxation_Energy,
    Decay_Length,
    run_But
]

parameters_resfind = [    
    Resonance_Finder,
    No_Homogenous_Grid_Points,
    No_Points_per_res,
    No_Points_per_EC_EF,
    Lancos_Iteration_Step_Size,
    Lancos_Iteration_Limit,
    Newton_Iteration_Step_Size,
    Newton_Solver_Convergance_Condition,
    run_But
]


    
    
    

# Combine all parameter lists into one
parameters = parameters_basic + parameters_domains + parameters_advanced + parameters_resfind

# Container for the structure of the simulation widgets
container_structure = VBox(layout=Layout(width='auto', height='100%', border='1px solid rgba(0, 0, 1,0 )'))

# Set the children for the different tabs in the GUI
container_materials = VBox()
container_materials.children = parameters_domains
container_structure.children = parameters_basic
container_ambient = VBox()
container_ambient.children = parameters_advanced
container_resfind = VBox()
container_resfind.children = parameters_resfind


# Create the Tabs widget with the different sections
Tabs = Tab(
    layout=Layout(
        width='650px',  # Allow dynamic scaling
    )
)

ptab=widgets.Tab()
ptab.children= [widgets.Output() for _ in range(3)]
ptab.set_title(0, "Dimension")
ptab.set_title(1, "Doping")
ptab.set_title(2, "Mole Fraction X")
Tabs.children = [container_structure, container_materials, container_ambient,container_resfind]
Tabs.set_title(0, "Basic")
Tabs.set_title(1, "Multiscale Domains")
Tabs.set_title(2, "Advanced")
Tabs.set_title(3, "Resonance Finder")

# Create the output container widget for simulation results
output_cont = VBox(layout=Layout(height='auto', width='auto', border='1px solid rgba(0, 0, 0, 1)'))
output_cont.children = [w_output]  # Assuming w_output is your output widget

# Create the options container (always visible widgets + tabs)
opt = VBox()
opt.children = [always_visible_widgets, Tabs]

properties=VBox()
properties.children = [output_image_area,output_widget]

# Create the layout for the whole application
main_layout = HBox(layout=Layout(width='auto'))  # Use HBox to align widgets horizontally
main_layout.children = [opt,properties]

all = VBox(layout=Layout(height='auto', width='auto', border='1px solid rgba(0, 0, 0, 1)'))
all.children = [main_layout,output_cont,outputContainer]
# Display the layout
display(all)



Current values:
{'Dimension': {'Contact': 30.0, 'U': 5.0}, 'Doping': {'Contact': 3e+17, 'U': 5e+16}, 'Mole Fraction': {'Contact': 1.0, 'U': 0.16666666666666666}}


click
Called


In [81]:
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from IPython.display import display

def generate_headers(num_of_barriers):
    headers = ['Contact', 'U']
    for i in range(num_of_barriers):
        headers.append('B')
        headers.append('W')
    headers.pop()  # Remove the last 'W'
    headers.append('U')
    headers.append('Contact')
    return headers

def generate_widgets(num_of_barriers):
    headers = generate_headers(num_of_barriers)
    total_columns = len(headers) + 1  # +1 for row labels

    # Create a grid layout with 4 rows and total_columns columns
    grid = GridspecLayout(4, total_columns,grip_gap='0')

    # Set the column widths using 'auto' and ensure they take up equal space
    grid.layout.grid_template_columns = '50% ' + ' '.join(['auto'] * (total_columns - 1))  # First column fixed, others flexible

    # Place headers in the first row, starting from the second column
    for col, header in enumerate(headers, start=1):
        grid[0, col] = widgets.Label(
            value=header,
            layout=widgets.Layout(display='flex', justify_content='center', margin='0')
        )

    # Place row labels in the first column
    row_labels = ['Material', 'Dimension (nm)', 'Doping Mole Fraction x']
    for row, label in enumerate(row_labels, start=1):
        grid[row, 0] = widgets.Label(
            value=label,
            layout=widgets.Layout(display='flex', justify_content='flex-end', width='auto', margin='0')
            )
        print("test")
        

    # Add input widgets under each header with no margins and fixed width
    for col in range(1, total_columns):  # Start from 1 to skip the row labels column
        for row in range(1, 4):  # Rows 1 to 3 for input widgets
            grid[row, col] = widgets.Text(
                layout=widgets.Layout(width='100%', margin='0',grid_gap='0')  # 100% width of grid cell, no margin/padding
            )
            
            

    # Display the grid
    display(grid)

# Example usage
generate_widgets(6)


test
test
test


GridspecLayout(children=(Label(value='Contact', layout=Layout(display='flex', grid_area='widget001', justify_c…